# Weather forecast as an $AR(1)$ Model in pyMC3

In [ ]:
%matplotlib inline
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import arviz as az
import pandas as pd

plt.style.use('seaborn-darkgrid')
print('Running on PyMC3 v{}'.format(pm.__version__))

### Load weather data from Blindern

In [ ]:
weather_data = pd.read_csv('temperature_blindern.csv',header=None, index_col=0).rename(columns={1:'temperature'})
print(f'Date range from {weather_data.index.min()} to {weather_data.index.max()}')
weather_data.sample(10)

### Use the last 2 years

In [ ]:
y = np.array(weather_data)[-730:]
dates = weather_data.index[-730:]

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(dates, y);

Consider the following AR(1) process, initialized in the
infinite past:
$$
   y_t = \theta y_{t-1} + \epsilon_t,
$$
where $\epsilon_t \overset{iid}{\sim} {\cal N}(0,1)$.  Suppose you'd like to learn about $\theta$ from a a sample of observations $Y^T = \{ y_0, y_1,\ldots, y_T \}$. 
  
We choose $\cal N (0, 1)$ as our prior for $\theta$

### Prior for  $\theta$

In [ ]:
from scipy.stats import norm
x_axis = np.linspace(-5, 5, 101)
plt.figure(figsize=(15,10))
plt.plot(x_axis, norm.pdf(x_axis, loc=0, scale=1))
plt.title('Prior for Theta');

### Define model

In [ ]:
sigma = 1.0
with pm.Model() as ar_weather:
    beta = pm.Normal('beta', mu=0, sigma=sigma)
    likelihood = pm.AR('likelihood', beta, sigma=1.0, observed=y)

### MCMC sampling

In [ ]:
with ar_weather:
    trace_weather = pm.sample(1000, tune=3000, cores=2)

In [ ]:
print('Mean: {:5.3f}'.format(trace_weather['beta'].mean()))
print('Std: {:5.3f}'.format(trace_weather['beta'].std()))

In [ ]:
az.plot_trace(trace_weather);

### Draw $\theta$ from posterior and predict tomorrows weather 

In [ ]:
beta_weather = np.random.choice(trace_weather['beta'], 1000, replace=False)
noise = np.random.normal(0, 1, 1000)
y_next = beta_weather * y[-1]   + noise

In [ ]:
plt.hist(y_next);
plt.xlabel('Temperature', fontsize=16);

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,10))
days = np.arange(0, len(y))
ax.plot(y);
ax.violinplot(y_next, [len(y)], widths=20, showmeans=True, showextrema=False, showmedians=True, points=500);
ax.set_xticks(np.arange(0, len(y), 30));
ax.set_xticklabels(dates[::30], rotation=60);
ax.set_xlabel('Date', fontsize=18)
ax.set_ylabel('Temperature in degrees celcius', fontsize=18)